In [44]:
import statsmodels.api as sm
import pandas as pd
import numpy as np

In [45]:
import warnings
warnings.filterwarnings('ignore')

In [71]:
pca = 'pca_'

df = pd.read_csv('datasets/train_set_pca.csv')
test_df = pd.read_csv('datasets/test_set_pca.csv')

In [72]:

if pca == 'pca_':
    X = df.loc[:, 'pca_0':]
else:
    X = df.loc[:, 'month':]
    y = df['cpi_pct']


X_test = test_df.loc[:, X.columns]

In [48]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso, Ridge, LinearRegression

In [49]:
# first we gonna build 2 benchark Models
# the first is just gonna predict the mean of the test target variable
# the second is just gonna predict the previous month cpi

In [37]:
pci_mean = y.mean()

y_test_pred = pd.DataFrame(pci_mean, columns=['pred'], index=range(X_test.shape[0]))
# y_test_pred.to_csv('predictions/benchmark1_mean.csv')

for the second model we already saved the prev cpi value as our test prediction:

`test_df[['cpi_lag1']].rename({'cpi_lag1': 'pred'}, axis=1).reset_index(drop=True).to_csv('predictions/benchmark2_prev.csv')`

In [38]:
# Shuffling lead to very high lambda values wich basicly meant moth models were just predicting mean (y intercept)
# X = X.sample(frac=1)

In [39]:
model = LinearRegression()

model.fit(X, y)

pred = model.predict(X_test)

pd.DataFrame(pred, index=X_test.index, columns=['pred']).to_csv(f'predictions/{pca}ols.csv') 

In [50]:
model = Lasso(max_iter=3000)

params = {
    'alpha': np.logspace(-8, -0.5, 100).tolist()
}

cv = GridSearchCV(model, params, cv=10)
cv.fit(X, y)

print(cv.best_estimator_)

pred = cv.predict(X_test)

# pd.DataFrame(pred, index=X_test.index, columns=['pred']).to_csv(f'predictions/{pca}lasso_cv.csv')

KeyboardInterrupt: 

In [41]:
model = Ridge()

params = {
    'alpha': np.linspace(1, 10000, 100).tolist()
}

cv = GridSearchCV(model, params, cv=10)
cv.fit(X, y)

print(cv.best_estimator_)

pred = cv.predict(X_test)

pd.DataFrame(pred, index=X_test.index, columns=['pred']).to_csv(f'predictions/{pca}ridge_cv.csv')

Ridge(alpha=2829.0)


In [42]:
# shuffle Train dataset for CV

In [74]:
model = Lasso(max_iter=3000)

df = pd.read_csv('datasets/train_set_pca.csv').sample(frac=1)

X = df.loc[:, 'pca_0':]
y = df['cpi_pct']

params = {
    'alpha': np.logspace(-8, -0.5, 100).tolist()
}

cv = GridSearchCV(model, params, cv=10)
cv.fit(X, y)

print(cv.best_estimator_)

pred = cv.predict(X_test)

pd.DataFrame(pred, index=X_test.index, columns=['pred']).to_csv(f'predictions/pca_lasso_cvshuffled.csv')

Lasso(alpha=8.697490026177834e-05, max_iter=3000)


In [62]:
lasso_cv_coefs = pd.DataFrame(cv.best_estimator_.coef_, index=cv.feature_names_in_, columns=['coef'])
lasso_cv_coefs.to_csv('models/lasso_cv_coefs.csv')

# Coeficients chosen by our best model

In [63]:
lasso_cv_coefs[lasso_cv_coefs.coef != 0]

,coef
month,-0.000022
jan-sep,0.000138
cpi_lag1,0.000537
usd_3m_pct,-0.000027
usd_3m_pct_6m_lag,-0.000105
usd_raw,-0.000252
wti_3m_pct,0.000922
wti_9m_pct,0.000051
wti_9m_pct_9m_lag,-0.000015
gas_3m_pct,0.000188
